#**Real-Time Face Mask Detection Using Deep Learning and Convolutional Neural Networks**

#### **Abstract:**
This focuses on the development and implementation of a Convolutional Neural Network (CNN)-based system to classify whether a person is wearing a face mask or not. Given the critical importance of face mask compliance in public health and safety, the proposed solution aims to offer a robust, real-time classification model using deep learning techniques. The project investigates various CNN architectures and optimizes the model for high accuracy and efficiency in deployment. Furthermore, techniques such as data augmentation, transfer learning, and model fine-tuning are explored to enhance performance on real-world datasets, ensuring scalability and effectiveness in different environments.

#**Reading data from the kaggle**

In [ ]:
# prompt: read kaggle datasets download -d belsonraja/face-mask-dataset-with-and-without-mask

!pip install kaggle

# Upload your kaggle.json file
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


#**Read data from the kaggle**

In [ ]:

!kaggle datasets download -d belsonraja/face-mask-dataset-with-and-without-mask


Dataset URL: https://www.kaggle.com/datasets/belsonraja/face-mask-dataset-with-and-without-mask
License(s): unknown
face-mask-dataset-with-and-without-mask.zip: Skipping, found more recently modified local copy (use --force to force download)


#**Unziping the code**

In [ ]:
!unzip /content/face-mask-dataset-with-and-without-mask.zip


Archive:  /content/face-mask-dataset-with-and-without-mask.zip
replace face_detector/deploy.prototxt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#read /content/facemask-dataset/dataset/with_mask and /content/facemask-dataset/dataset/without_mask
import os

with_mask_dir = '/content/facemask-dataset/dataset/with_mask'
without_mask_dir = '/content/facemask-dataset/dataset/without_mask'

with_mask_files = os.listdir(with_mask_dir)
without_mask_files = os.listdir(without_mask_dir)


In [ ]:
print("Files in with_mask directory:")
for file in with_mask_files:
  print(file)

In [ ]:
print("\nFiles in without_mask directory:")
for file in without_mask_files:
  print(file)

In [ ]:
len(with_mask_files)

In [ ]:
len(without_mask_files)

In [ ]:
#randomly get images from with_mask_files and without_mask_files and plot them

import matplotlib.pyplot as plt
import random
import cv2

#Choose a random number of images to display (e.g., 5)
num_images_to_display = 2

#Get a random sample of image filenames from each directory
random_with_mask_files = random.sample(with_mask_files, num_images_to_display)
random_without_mask_files = random.sample(without_mask_files, num_images_to_display)

In [ ]:
#Function to display images in a grid
def display_images(image_paths, title):
  plt.figure(figsize=(15, 5))
  plt.suptitle(title)
  for i, image_path in enumerate(image_paths):
    img = cv2.imread(os.path.join(with_mask_dir, image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(img)
    plt.axis('off')
  plt.show()

In [ ]:
# Display images with mask
display_images([os.path.join(with_mask_dir, filename) for filename in random_with_mask_files], "Images with Mask")

In [ ]:
# Display images without mask
display_images([os.path.join(without_mask_dir, filename) for filename in random_without_mask_files], "Images without Mask")


In [ ]:
# make a cnn model to ditinguish  btween
# with_mask_files
# without_mask_files image and test the accuracy

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Define image dimensions
img_width, img_height = 150, 150

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into training and validation sets
)

# Load and preprocess the data
train_generator = train_datagen.flow_from_directory(
    '/content/facemask-dataset/dataset',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/facemask-dataset/dataset',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


#**creating CNN model to predict the masked or non-masked faces**

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=20,  # You can adjust the number of epochs
    validation_data=validation_generator
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


66/96 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.7143 - loss: 0.6055

#**Accuracy of the model**

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print('Validation accuracy:', accuracy)


In [ ]:
#using above model i want to pass an images to know wehter it has mask or not

import numpy as np
from tensorflow.keras.preprocessing import image

def predict_mask(image_path):
  img = image.load_img(image_path, target_size=(img_width, img_height))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.

  prediction = model.predict(img_array)

  if prediction[0][0] > 0.5:
    print("The image likely does not have a mask.")
  else:
    print("The image likely has a mask.")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the image
def show_image(image_path):
    img = mpimg.imread(image_path)  # Load the image from the given path
    plt.imshow(img)                 # Display the image using matplotlib
    plt.axis('off')                 # Hide the axis
    plt.show()                      # Render the image

#**Prediction using the model on a new image**

In [ ]:
# Example usage
image_path = '/content/lp_image.jpeg'  # Replace with the actual image path
show_image(image_path)
predict_mask(image_path_to_predict)

In [ ]:
# Example usage
image_path = '/content/befunky_2022-6-0_20-20-3.jpeg'  # Replace with the actual image path
show_image(image_path)
predict_mask('/content/lp_image.jpeg')